Careless responder feature engineering and analysis (inital focus on psych_flex survey only)

In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import scipy as sp

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import silhouette_score
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
data_path = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/PeskindTBI/merge_data_first.csv'

In [ ]:
#read in csv containing data from all surveys
data = pd.read_csv(data_path)
data = pd.DataFrame(data = data)

print('Original data shape:\n', data.shape, '\n')
data.head(1)

In [ ]:

variables_long = ['ScreenAge',
       'cestotal', 'MnthSncBlst', 'QEDist_sum',
       'QEDist_mean', 'NSITot', 'TBITot', 'hrslp', 'PSQItot', 'PCLTot',
       'CAPSTotal', 'PHQTot', 'PTSD_YN', 
       'QKOIorA', 'QKOExpMil', 'QKOAllMil', 'QKOLife', 'QBlstExp',
        'tbiDizzy', 'tbiBalan', 'tbiCoord', 'tbiHeada', 'tbiNaus',
       'tbiVision', 'tbiLight', 'tbiHear', 'tbiNoise', 'tbiTingl',
       'tbiTstsml', 'tbiAppet', 'tbiConc', 'tbiForget', 'tbiDecis',
       'tbiSlow', 'tbiEnergy', 'tbiSleep', 'tbiAnx', 'tbiSad', 'tbiIrrit',
       'tbiOverw', 'tbiDisin', 'tbiWithd', 'tbiRing', 'tbiMoods',
       'tbiFight', 'tbiSpch', 'DA', 'DOPA', 'NE', 'QBEACRM', 'BISAtt', 'BISMtr', 'BISNonpl', 'BISTot', 'DvpHA', 'DvpBP',
        'Frontal_Mid_l', 'Frontal_Mid_r', 'Insula_l', 'Insula_r',
       'Cingulum_Ant_l', 'Cingulum_Ant_r', 'Amygdala_l', 'Amygdala_r',
       'CaudateNucl_l', 'CaudateNucl_r', 'Putamen_l', 'Putamen_r',
       'Pallidum_l', 'Pallidum_r', 'Medulla', 'Midbrain', 'Pons', 'auditc', 
       'AUDIT1', 'AUDIT2', 'AUDIT3']

variables = ['cestotal', 'QEDist_sum', 'QEDist_mean',  
       'QKOIorA', 'QKOExpMil', 'QKOAllMil', 'QKOLife', 'QBlstExp']

data_TBI = data[data['Group'] == 'T']
data_TBI.set_index('TBIID', inplace=True)
data_TBI_variables = data_TBI[variables].dropna()
print(data_TBI_variables.shape)
data_TBI_variables.head(1)

In [ ]:
# center and scale the data
scaler = StandardScaler()

data_TBI_variables_scaled = scaler.fit_transform(data_TBI_variables)

In [ ]:
k_range = range(2,20)
scores = []
for k in k_range:
    km_ss = KMeans(n_clusters=k, random_state=1)
    km_ss.fit(data_TBI_variables_scaled)
    scores.append(silhouette_score(data_TBI_variables_scaled, km_ss.labels_))

# plot the results
plt.plot(k_range, scores)
plt.title('data_TBI_variables clustering')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')

In [ ]:
km_3 = KMeans(n_clusters=2,random_state=1234)
km_3.fit(data_TBI_variables_scaled)
data_TBI_variables['kmeans_cluster'] = ["cluster_" + str(label) for label in km_3.labels_ ]
data_TBI_variables.head()

In [ ]:
data_TBI_variables['kmeans_cluster'].value_counts()

In [ ]:
#add cluster column to PF survey df
participants = data_TBI.index

for part in participants:
    try:
        data_TBI.loc[data_TBI.index == part, 'kmeans_cluster'] = data_TBI_variables[data_TBI_variables.index == part]['kmeans_cluster'].values
    except:
        data_TBI.loc[data_TBI.index == part, 'kmeans_cluster'] = np.nan
data_TBI.head()

In [ ]:
data_TBI_cluster = data_TBI.groupby('kmeans_cluster')['AUDITtot'].value_counts() 
data_TBI_cluster_perc = data_TBI.groupby('kmeans_cluster')['AUDITtot'].value_counts() / data_TBI.groupby('kmeans_cluster')['AUDITtot'].count() 
data_TBI_cluster_perc.to_csv('data_TBI_cluster_perc.csv')

In [ ]:
#plt.style.use('seaborn-white')
for variable in variables_long:
    plt.figure(figsize=(7,7))
    g = sns.barplot(x='kmeans_cluster', y=variable, data=data_TBI, ci=68, palette="rocket", order=["cluster_0", "cluster_1"])
    plt.show()

In [ ]:
tsne = TSNE(n_components=2, random_state=1234)
tsne_features = tsne.fit_transform(data_TBI_variables_scaled)

print(tsne_features.shape)
tsne_df = pd.DataFrame(data = tsne_features, columns = ['tsne_0', 'tsne_1'], index = data_TBI_variables.index)
data_TBI_variables = pd.concat([data_TBI_variables, tsne_df], axis = 1)

In [ ]:
pca_ss = KernelPCA(n_components=2, kernel='linear', random_state=1)
kernel_pca_linear = pca_ss.fit_transform(data_TBI_variables_scaled)

print(kernel_pca_linear.shape)

kernel_pca_linear_df = pd.DataFrame(data = kernel_pca_linear, columns = ['pca_linear_0', 'pca_linear_1'], index = data_TBI_variables.index)
data_TBI_variables = pd.concat([data_TBI_variables, kernel_pca_linear_df], axis = 1)

In [ ]:
pca_ss = KernelPCA(n_components=2, kernel='rbf', random_state=1)
kernel_pca_rbf = pca_ss.fit_transform(data_TBI_variables_scaled)

print(kernel_pca_rbf.shape)

kernel_pca_rbf_df = pd.DataFrame(data = kernel_pca_rbf, columns = ['pca_rbf_0', 'pca_rbf_1'], index = data_TBI_variables.index)
data_TBI_variables = pd.concat([data_TBI_variables, kernel_pca_rbf_df], axis = 1)

In [ ]:
for variable in variables:
    plt.figure(figsize=(7,7))
    sns.scatterplot(x = 'pca_rbf_0', y = 'pca_rbf_1', data = data_TBI_variables, hue=variable)
    plt.show()
#plt.title("Projection of the data on 2 components + ground truth labels")